# 내 코드

In [19]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [20]:
from langchain_core.documents import Document
from langchain.tools import tool
from langchain_openai import ChatOpenAI #bind_tools를 위해서 pdf와 달리 이렇게 수정해 주어야 함.

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [13]:
animal_facts = {
    "강아지" : "강아지는 인간과 가장 친한 반려동물 중 하나입니다. 충성심이 강하고 활발합니다.",
    "고양이" : "고양이는 독립적인 성격을 가진 반려동물로, 깨긋하고 조용한 성격을 가졌습니다.",
    "코끼리" : "코끼리는 육상 동물 중 가장 큰 동물로, 매우 높은 지능과 사회성을 지닙니다.",
    "기린" : "기린은 목이 매우 길며, 주로 아프리카 사바나에서 서식합니다."
}

In [14]:
animal_docs = []

for key, value in animal_facts.items():
    animal_doc = Document(
        page_content= key + value,
    )
    animal_docs.append(animal_doc)

print(animal_docs)


[Document(metadata={}, page_content='강아지강아지는 인간과 가장 친한 반려동물 중 하나입니다. 충성심이 강하고 활발합니다.'), Document(metadata={}, page_content='고양이고양이는 독립적인 성격을 가진 반려동물로, 깨긋하고 조용한 성격을 가졌습니다.'), Document(metadata={}, page_content='코끼리코끼리는 육상 동물 중 가장 큰 동물로, 매우 높은 지능과 사회성을 지닙니다.'), Document(metadata={}, page_content='기린기린은 목이 매우 길며, 주로 아프리카 사바나에서 서식합니다.')]


In [ ]:
embeddings_model = OpenAIEmbeddings(
    model = "text-embedding-3-small"
)

animal_db = Chroma.from_documents(
    documents=animal_docs,
    embedding=embeddings_model,
    collection_name="animal_facts",
    persist_directory="./chroma_db"
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [21]:
animal_retriever = animal_db.as_retriever(
    search_kwargs={"k": 1},
)

query = "강아지는 어떤 동물이야?"
doc = animal_retriever.invoke(query)
print(doc)

[Document(metadata={}, page_content='강아지강아지는 인간과 가장 친한 반려동물 중 하나입니다. 충성심이 강하고 활발합니다.')]


In [22]:
@tool
def search_animal(query : str) -> list[Document]:
    """
    이름에 해당한느 동물에 대한 정보와 설명을 간단하게 반환합니다.
    """

    docs = animal_retriever.invoke(query)
    if len(docs) == 1:
        return docs
    
    return [Document(page_content="해당 동물의 정보를 찾을 수 없습니다.")]

In [23]:
prompt = ChatPromptTemplate([
    ("system", f"You are helpful AI assistant which explains about the animal."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[search_animal])
llm_chain = prompt | llm_with_tools

In [24]:
#여기서 config를 왜 쓰는 걸까?

@chain
def search_animal_chain(user_input:str, config : RunnableConfig):
    input_ = {"user_input" : user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        if tool_call["name"] == "search_animal":
            tool_message = search_animal.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

    return llm_chain.invoke({**input_, "messages" : [ai_msg, *tool_msgs]}, config=config)

response = search_animal_chain.invoke("코끼리에 대해서 알려줘.")

print(response.content)

코끼리는 육상 동물 중 가장 큰 동물로, 매우 높은 지능과 사회성을 지닙니다. 이들은 주로 아프리카와 아시아에 서식하며, 가족 단위로 생활하는 경향이 있습니다. 코끼리는 긴 코와 큰 귀가 특징이며, 이들은 환경에 적응하고 서로 소통하는 데 중요한 역할을 합니다.


# 교수님 코드

In [ ]:
from langchain_core.documents import Document
from langchain.tools import tool
from langchain_openai import ChatOpenAI #bind_tools를 위해서 pdf와 달리 이렇게 수정해 주어야 함.

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


In [27]:
animal_facts = {
    "강아지" : "강아지는 인간과 가장 친한 반려동물 중 하나입니다. 충성심이 강하고 활발합니다.",
    "고양이" : "고양이는 독립적인 성격을 가진 반려동물로, 깨긋하고 조용한 성격을 가졌습니다.",
    "코끼리" : "코끼리는 육상 동물 중 가장 큰 동물로, 매우 높은 지능과 사회성을 지닙니다.",
    "기린" : "기린은 목이 매우 길며, 주로 아프리카 사바나에서 서식합니다."
}

In [28]:
@tool
def search_animal(query: str) -> list[Document]:
    """
    이름에 해당하는 동물 정보를 간단히 반환합니다.
    """

    if query in animal_facts:
        return [Document(page_content=animal_facts[query])]
    return [Document(page_content="해당 동물 정보를 찾을 수 없습니다.")]

In [31]:
llm = ChatOpenAI(
    model = "gpt-4o-mini",
)

llm_with_tools = llm.bind_tools(tools=[search_animal])

In [ ]:
from pprint import pprint

question = "고양이에 대해 알려줘."
ai_msg = llm_with_tools.invoke(question)

pprint(ai_msg)
print("-"*50)

pprint(ai_msg.content) 
print("-"*50)

pprint(ai_msg.tool_calls) # 어떻게 쿼리가 동물 이름으로만 나온다는 것을 확신하지?
print("-"*50)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_feTs0lYNZBAhYZRFpUsvOz6q', 'function': {'arguments': '{"query":"고양이"}', 'name': 'search_animal'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 58, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9e6c6c38-1106-467e-8d2e-163b6eaba652-0', tool_calls=[{'name': 'search_animal', 'args': {'query': '고양이'}, 'id': 'call_feTs0lYNZBAhYZRFpUsvOz6q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 17, 'total_tokens': 75})
--------------------------------------------------
''
-------------------------------------------------

In [38]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate

today = datetime.today().strftime("%Y-%m-%d")

prompt = ChatPromptTemplate([
    ("system", f"You are helpful AI assistant. Today is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{message}"), #tool에 대한 정보를 넣음. + tool이 리턴한 결과
])

llm = ChatOpenAI(model='gpt-4o-mini')

llm_with_tools = llm.bind_tools(tools=[search_animal])

llm_chain = prompt | llm_with_tools

@chain
def animal_search_chain(user_input: str):
    input_ = {"user_input" : user_input}
    ai_msg = llm_chain.invoke(input_)

    pprint(ai_msg)

    tool_msg = search_animal.batch(ai_msg.tool_calls) #이 부분에서 문제?

    pprint(tool_msg)

    return llm_chain.invoke({**input_, "message" : [ai_msg, *tool_msg]})

response = animal_search_chain.invoke("고양이에 대해서 알려줘")
pprint(response.content)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0hliXUOgvwSwzIB0dZZ5vQbL', 'function': {'arguments': '{"query":"고양이"}', 'name': 'search_animal'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 73, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--99ff7c7b-5b09-42fa-a540-d243448eada6-0', tool_calls=[{'name': 'search_animal', 'args': {'query': '고양이'}, 'id': 'call_0hliXUOgvwSwzIB0dZZ5vQbL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 73, 'output_tokens': 17, 'total_tokens': 90})
[ToolMessage(content="[Document(metadata={}, page_content='고양이는 독립적인 성격을 가진 반려동물로, 깨긋하고 조용한 성격을 가졌습니다.'